# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a classification example. This example is derived from [Hyperparameter Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use Tune to execute your training hyperparameter sweep, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#summary) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine or analyse the results returned
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Understanding Hyperparameter Tuning](https://github.com/anyscale/academy/blob/main/ray-tune/02-Understanding-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +0.7s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.3s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.4s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +1.4s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-27 07:52:37,909	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-27 07:52:37,941	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_01e5cdbe46829718.zip' (6.30MiB) to Ray cluster...
2022-01-27 07:52:43,149	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_01e5cdbe46829718.zip'.


(anyscale +15.9s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_jUg93ra8KHWTzAMZv5nig2Rb
(anyscale +15.9s) URL for head node of cluster: https://session-jug93ra8khwtzamzv5nig2rb.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial with a set of unique config from the search space
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [5]:
 # Specify the typical hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="60%" width="70%"> 

In [6]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    verbose=1,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=4844) == Status ==
(run pid=4844) Current time: 2022-01-27 07:53:03 (running for 00:00:00.22)
(run pid=4844) Memory usage on this node: 2.4/62.0 GiB
(run pid=4844) Using FIFO scheduling algorithm.
(run pid=4844) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.22 GiB heap, 0.0/92.18 GiB objects
(run pid=4844) Result logdir: /home/ray/ray_results/train_func_model_2022-01-27_07-53-02
(run pid=4844) Number of trials: 4/4 (4 PENDING)
(run pid=4844) 
(run pid=4844) 


(ImplicitFunc pid=4953) 2022-01-27 07:53:05,230	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=4953) 2022-01-27 07:53:06,947	INFO main.py:1024 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=5015) [07:53:06] task [xgboost.ray]:139780947481888 got new rank 0
(_RemoteRayXGBoostActor pid=5017) [07:53:06] task [xgboost.ray]:140615337628960 got new rank 2
(_RemoteRayXGBoostActor pid=5018) [07:53:06] task [xgboost.ray]:140027524908320 got new rank 3
(_RemoteRayXGBoostActor pid=5016) [07:53:06] task [xgboost.ray]:140220007744848 got new rank 1
(ImplicitFunc pid=4953) 2022-01-27 07:53:07,863	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 2.76 seconds (0.91 pure XGBoost training time).


(run pid=4844) == Status ==
(run pid=4844) Current time: 2022-01-27 07:53:12 (running for 00:00:09.90)
(run pid=4844) Memory usage on this node: 2.3/62.0 GiB
(run pid=4844) Using FIFO scheduling algorithm.
(run pid=4844) Resources requested: 15.0/80 CPUs, 0/0 GPUs, 0.0/216.22 GiB heap, 0.0/92.18 GiB objects
(run pid=4844) Current best trial: 3576b_00001 with train-error=0.010545 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0009316428296060249, 'subsample': 0.9212008867068575, 'max_depth': 5, 'nthread': 1, 'n_jobs': 1}
(run pid=4844) Result logdir: /home/ray/ray_results/train_func_model_2022-01-27_07-53-02
(run pid=4844) Number of trials: 4/4 (3 RUNNING, 1 TERMINATED)
(run pid=4844) 
(run pid=4844) 


(ImplicitFunc pid=445, ip=172.31.126.32) 2022-01-27 07:53:13,050	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=445, ip=172.31.115.123) 2022-01-27 07:53:13,052	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=444, ip=172.31.121.3) 2022-01-27 07:53:13,053	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=445, ip=172.31.115.123) 2022-01-27 07:53:14,768	INFO main.py:1024 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=481, ip=172.31.115.123) [07:53:14] task [xgboost.ray]:140145258268032 got new rank 1
(_RemoteRayXGBoostActor pid=482, ip=172.31.115.123) [07:53:14] task [xgboost.ray]:140158640801152 got new rank 2
(_RemoteRayXGBoostActor pid=483, ip=172.31.115.123) [07:53:14] task [xgboost.ray]:139818421065088 got

(run pid=4844) == Status ==
(run pid=4844) Current time: 2022-01-27 07:53:14 (running for 00:00:11.90)
(run pid=4844) Memory usage on this node: 2.3/62.0 GiB
(run pid=4844) Using FIFO scheduling algorithm.
(run pid=4844) Resources requested: 15.0/80 CPUs, 0/0 GPUs, 0.0/216.22 GiB heap, 0.0/92.18 GiB objects
(run pid=4844) Current best trial: 3576b_00001 with train-error=0.010545 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0009316428296060249, 'subsample': 0.9212008867068575, 'max_depth': 5, 'nthread': 1, 'n_jobs': 1}
(run pid=4844) Result logdir: /home/ray/ray_results/train_func_model_2022-01-27_07-53-02
(run pid=4844) Number of trials: 4/4 (3 RUNNING, 1 TERMINATED)
(run pid=4844) 
(run pid=4844) 
(run pid=4844) 2022-01-27 07:53:15,799	INFO commands.py:292 -- Checking External environment settings
(run pid=4844) 2022-01-27 07:53:16,892	WARN util.py:136 -- The `head_node` field is deprecated and will be ignored. 

(ImplicitFunc pid=445, ip=172.31.115.123) 2022-01-27 07:53:19,775	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 6.87 seconds (5.00 pure XGBoost training time).


(run pid=4844) 2022-01-27 07:53:19,840	WARN commands.py:269 -- Loaded cached provider configuration
(run pid=4844) 2022-01-27 07:53:19,841	WARN commands.py:270 -- If you experience issues with the cloud provider, try re-running the command with --no-config-cache.
(run pid=4844) 2022-01-27 07:53:20,854	INFO command_runner.py:357 -- Fetched IP: 172.31.121.3
(run pid=4844) 2022-01-27 07:53:20,854	INFO log_timer.py:25 -- NodeUpdater: ins_TQHm3Z2WMkpsAvn5s5pECCdp: Got IP  [LogTimer=32ms]
(run pid=4844) == Status ==
(run pid=4844) Current time: 2022-01-27 07:53:21 (running for 00:00:18.79)
(run pid=4844) Memory usage on this node: 2.4/62.0 GiB
(run pid=4844) Using FIFO scheduling algorithm.
(run pid=4844) Resources requested: 15.0/80 CPUs, 0/0 GPUs, 0.0/216.22 GiB heap, 0.0/92.18 GiB objects
(run pid=4844) Current best trial: 3576b_00001 with train-error=0.010545 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.00093164282

(ImplicitFunc pid=445, ip=172.31.126.32) 2022-01-27 07:53:23,689	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 10.78 seconds (8.82 pure XGBoost training time).
(ImplicitFunc pid=444, ip=172.31.121.3) 2022-01-27 07:53:23,684	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 10.84 seconds (8.90 pure XGBoost training time).
(run pid=4844) 2022-01-27 07:53:23,865	INFO tune.py:626 -- Total run time: 21.73 seconds (20.75 seconds for the tuning loop).


(run pid=4844) == Status ==
(run pid=4844) Current time: 2022-01-27 07:53:23 (running for 00:00:20.75)
(run pid=4844) Memory usage on this node: 2.4/62.0 GiB
(run pid=4844) Using FIFO scheduling algorithm.
(run pid=4844) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.22 GiB heap, 0.0/92.18 GiB objects
(run pid=4844) Current best trial: 3576b_00001 with train-error=0.010545 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0009316428296060249, 'subsample': 0.9212008867068575, 'max_depth': 5, 'nthread': 1, 'n_jobs': 1}
(run pid=4844) Result logdir: /home/ray/ray_results/train_func_model_2022-01-27_07-53-02
(run pid=4844) Number of trials: 4/4 (4 TERMINATED)
(run pid=4844) 
(run pid=4844) 


In [7]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0009316428296060249, 'subsample': 0.9212008867068575, 'max_depth': 5}


In [8]:
analysis.results_df.head(5)

/usr/local/anaconda3/envs/ray-core/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
3576b_00000,0.598461,0.017575,0.003546,True,None,None,10,66b1b985980f4f8192ae598098438b6d,2022-01-27_07-53-23,1643298803,...,10,"0_eta=0.012158,max_depth=4,subsample=0.75912",approx,binary:logistic,"[logloss, error]",0.012158,0.759123,4,1,1
3576b_00001,0.684981,0.010545,0.003856,True,None,None,10,be3ee5276c5c4ec6a181b21038a00859,2022-01-27_07-53-07,1643298787,...,10,"1_eta=0.00093164,max_depth=5,subsample=0.9212",approx,binary:logistic,"[logloss, error]",0.000932,0.921201,5,1,1
3576b_00002,0.691071,0.029877,0.003715,True,None,None,10,82cc3b9a582e4ed6b7572248e0170484,2022-01-27_07-53-19,1643298799,...,10,"2_eta=0.00024877,max_depth=3,subsample=0.60588",approx,binary:logistic,"[logloss, error]",0.000249,0.605882,3,1,1
3576b_00003,0.692246,0.028120,0.003723,True,None,None,10,b081c547f16b427c8e484a6e149b512a,2022-01-27_07-53-23,1643298803,...,10,"3_eta=0.00010676,max_depth=7,subsample=0.55468",approx,binary:logistic,"[logloss, error]",0.000107,0.554683,7,1,1


---

In [9]:
ray.shutdown()

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)